In [1]:
! pip install kaggle

uploading kaggle json file

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
#using API to fetch the datset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 80% 65.0M/80.9M [00:00<00:00, 109MB/s] 
100% 80.9M/80.9M [00:00<00:00, 109MB/s]


In [5]:
#extracting the compressed dataset  from zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

#opening the file and extract
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


In [6]:
#Importing the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [7]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
#data processing
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [9]:
twitter_data.shape


(1599999, 6)

In [10]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#naming the columns and reading the dataset
column_name = ['target','id','date','flag','user','text']

In [12]:

twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_name ,encoding = 'ISO-8859-1')

In [13]:
twitter_data.shape

(1600000, 6)

In [14]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
#checking missing values
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [16]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
twitter_data.replace({'target':{4:1}},inplace=True)

**Stemming**

eg : actor,actress,acting = act (Root word)

In [21]:
port_stem = PorterStemmer()

In [22]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [23]:
twitter_data['stemmed_content']= twitter_data['text'].apply(stemming)

In [24]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [25]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [26]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [29]:
#separating the data and the label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [30]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [31]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into train and test data

In [33]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,stratify=Y,random_state= 4)

In [34]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1120000,) (480000,)


In [35]:
print(Y.shape,Y_train.shape,Y_test.shape)

(1600000,) (1120000,) (480000,)


In [36]:
#Converting the textual data to numerical data- we will use a vectoriser Tfidfvectorizer
vectorizer= TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [37]:
print(X_train)

  (0, 114988)	0.47995077500359007
  (0, 373478)	0.3682048299198392
  (0, 95787)	0.7962866674741926
  (1, 265707)	0.3640353724954491
  (1, 400931)	0.3599230314585979
  (1, 401434)	0.42237129302182114
  (1, 326374)	0.7480214902185818
  (2, 399216)	0.43186839049549774
  (2, 355248)	0.26047060402354455
  (2, 146306)	0.27807866014469496
  (2, 77459)	0.2883681325114934
  (2, 255893)	0.22069688591515554
  (2, 372586)	0.23568995069404086
  (2, 365557)	0.27217209846028295
  (2, 281037)	0.6378293793510319
  (3, 391074)	0.4534094467006167
  (3, 324840)	0.8913023469298398
  (4, 140632)	0.3999862276341077
  (4, 12302)	0.9165211496212381
  (5, 321407)	0.43414751289454606
  (5, 215289)	0.14386715488741114
  (5, 220678)	0.17026963391740532
  (5, 321691)	0.29449731546397395
  (5, 115106)	0.26863503766089736
  (5, 349040)	0.5133867944667918
  :	:
  (1119996, 400026)	0.1494111376412366
  (1119996, 367402)	0.1478438745565229
  (1119996, 143410)	0.18535351481167048
  (1119997, 25859)	0.5926416738796091
  (

In [38]:
print(X_test)

  (0, 416014)	0.27972400042298307
  (0, 400492)	0.7533779305052855
  (0, 221605)	0.30293559090565864
  (0, 143992)	0.36028891590795603
  (0, 138278)	0.2767668201331842
  (0, 119378)	0.23663945048282053
  (1, 419622)	0.4788377072062665
  (1, 402724)	0.3632219819799668
  (1, 220678)	0.2952521567490841
  (1, 156025)	0.375704033981877
  (1, 85358)	0.46018832398264725
  (1, 73969)	0.44573937588998286
  (2, 416500)	0.28720835176115517
  (2, 401348)	0.22398788479001505
  (2, 362115)	0.4942395019464537
  (2, 345538)	0.39197450813458756
  (2, 328692)	0.14447448661703577
  (2, 270035)	0.2388111177595396
  (2, 235382)	0.1899328504194884
  (2, 207101)	0.4207287214883508
  (2, 98581)	0.25529199123653606
  (2, 55372)	0.1788767101820872
  (2, 48031)	0.2173607575762857
  (2, 11817)	0.1844362780544421
  (3, 386002)	0.23630570575598214
  :	:
  (479995, 127218)	0.4942464249583358
  (479996, 400897)	0.8242203169847728
  (479996, 379586)	0.466388091597152
  (479996, 245598)	0.3211588658061412
  (479997, 38

In [39]:
#training the ML Model- logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [40]:
#model evaluation
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [41]:
print("Accuracy score on the training data :",training_data_accuracy)

Accuracy score on the training data : 0.8104107142857143


In [42]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [44]:
print("Accuracy score on the testing data :",testing_data_accuracy)

Accuracy score on the testing data : 0.77851875


Saving the trained model

In [45]:
import pickle

In [47]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [48]:
#Using a saved model for future predictions
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [54]:
X_new = X_test[700]
print(Y_test[700])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet
